In [1]:
import pandas as pd
import numpy as np

from my_functions_idealisedprofiles import *

### Get version of landcover array with just 'urban' and 'rural' categories

In [2]:
# Read in the data
landcover, out_meta = prepare_rainfall_scenario_raster(model_directory + "LandCover_clipped.tif", True)
# Convert the 1 and 6 values to 10 (for urban) and the rest to 11 (for non-urban).  
landcover_mod =  np.where(landcover==1, 10, landcover)
landcover_mod =  np.where(landcover_mod==6, 10, landcover_mod)
# Convert the rest of the classes to 11
for i in [1,2,3,4,5,7,8,9]:
    landcover_mod =  np.where(landcover_mod==i, 11, landcover_mod)

### Define the names of the method (shorter and longer versions)

In [121]:
methods =['6h_feh_sp', '6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp',
          '6h_sp_bl_0.6','6h_sp_bl_0.7','6h_sp_bl_0.8','6h_sp_bl_0.9']
short_ids = ['6h_feh_sp', '6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp',
          '6h_sp_bl_0.6','6h_sp_bl_0.7','6h_sp_bl_0.8','6h_sp_bl_0.9']   

### Find maximum intensity for each method and minute in which it occurs (to use in sorting results analysis)

In [71]:
maxs = []
min_of_maxs = []

# Add FEH data
feh_precip=pd.read_csv("../../CreateSyntheticRainfallEvents/ReFH2_singlepeak/6hr_100yrRP/PostLossRemoval/6h_feh_singlepeak_urban.csv")
maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
min_of_maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

for method in methods[1:]:
#     if method == '6h_single-peak':
#         precip=pd.read_csv("../../CreateSyntheticRainfallEvents/SyntheticProfiles/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(method))
#     else:
    precip=pd.read_csv("../../CreateSyntheticRainfallEvents/IdealisedProfiles/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(method))
    maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
    min_of_maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

### Create versions of lists of methods, in order based on max intensity and the the timing of the max intensity 

In [72]:
short_ids_by_loading=  pd.DataFrame({"min": min_of_maxs, 'method_name': short_ids}).sort_values('min')["method_name"].tolist()
short_ids_by_loading.remove('6h_feh_sp')
short_ids_by_loading = ['6h_feh_sp']+short_ids_by_loading

short_ids_by_intensity = pd.DataFrame({"min": maxs, 'method_name': short_ids}).sort_values('min', ascending = False)["method_name"].tolist()
short_ids_by_intensity.remove('6h_feh_sp')
short_ids_by_intensity = ['6h_feh_sp']+short_ids_by_intensity

### Create dataframe of colours for each cluster (based on their loading)

In [75]:
colours_df = create_colours_df(short_ids_by_loading, short_ids)

### Create list of filepaths, formatted to be used for either depth or velocity

In [124]:
fps = []
for method_num, short_id in enumerate(short_ids):
    if short_id == '6h_fp':
        fp = '../../../../FloodModelling/MeganModel_New/6h_fp/{} (Max).Resampled.Terrain.tif'
    else:
        fp = model_directory + "{}/{} (Max).Resampled.Terrain.tif".format(short_id, '{}')
    fps.append(fp)

### Define breaks for categorising velocity and depth

In [126]:
# Define breaks to split the depths/velocities on
breaks_depths = np.array([0, 0.3, 0.6, 1.2, 100])  
labels_depth = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
breaks_velocity = np.array([0,0.25,0.5,2,100])
labels_velocity = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]

# <u> Flood extent </u>
To examine whether the rainfall's temporal distribution influences the total extent of flooding, the number of flooded cells and the total flooded area in km2 (incl. only cells with depth >0.1m) is compared between the profile with a single peak, and the three methods for producing multi-peaked rainfall events. b

### Create dataframes containing the (total/urban) flooded area in each depth/velocity bin

In [128]:
velocity_counts, velocity_props = create_binned_counts_and_props(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values)
depth_counts, depth_props = create_binned_counts_and_props(fps, 'Depth', breaks_depths, labels_depth, remove_little_values)

In [131]:
velocity_counts_urban, velocity_props_urban = create_binned_counts_and_props_urban(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values, landcover_mod)
depth_counts_urban, depth_props_urban = create_binned_counts_and_props_urban(fps, 'Depth', breaks_depths, labels_depth, remove_little_values, landcover_mod)

### Create dataframes containing the (total/urban) flooded area

In [132]:
totals_df = create_totals_df(velocity_counts)
totals_df_urban = create_totals_df(velocity_counts_urban)      

### Create dataframes containing the % diff in the flooded area between single peak and each other method  

In [133]:
percent_diffs_df = find_percentage_diff (totals_df, fps) 
percent_diffs_df_urban = find_percentage_diff (totals_df_urban, fps)   

## Find number of cells in which each method leads to the worst flooding (depth/velocity)

In [137]:
# Find the number of flooded cells with the worst flooding for each method
worst_case_method_depth = find_worst_case_method(fps, short_ids, 'Depth')
worst_case_method_velocity = find_worst_case_method(fps, short_ids,  'Velocity') 

In [139]:
# Remove multiple matches and nan
worst_case_method_depth = worst_case_method_depth[~worst_case_method_depth['values'].isin(['multiple matches','nan'])]
worst_case_method_velocity = worst_case_method_velocity[~worst_case_method_velocity['values'].isin(['multiple matches','nan'])]

# # Reorder (and also add in the methods that are missing)
worst_case_method_depth = pd.merge(worst_case_method_depth,  pd.DataFrame({'values': short_ids}), how="outer")
worst_case_method_depth = worst_case_method_depth.reindex(worst_case_method_depth['values'].map(dict(zip(short_ids, range(len(short_ids))))).sort_values().index)
worst_case_method_depth.reset_index(inplace=True,drop=True)

worst_case_method_velocity = pd.merge(worst_case_method_velocity,  pd.DataFrame({'values': short_ids}), how="outer")
worst_case_method_velocity = worst_case_method_velocity.reindex(worst_case_method_velocity['values'].map(dict(zip(short_ids, range(len(short_ids))))).sort_values().index)
worst_case_method_velocity.reset_index(inplace=True,drop=True)

## Find number of cells with each hazard rating

In [144]:
hazard_counts, hazard_props = create_binned_counts_and_props_hazard(fps)

## Find number of cells which have moved between hazard categories

In [146]:
hazard_cat_changes = create_binned_counts_and_props_hazard_cat_change(fps)

### Create a dataframe containing all the info on each of the scenarios

In [147]:
cluster_results = pd.DataFrame({'Cluster_num': short_ids, "MaxRainfallIntensity": maxs,  
    "MaxRainfallIntensityMinute": min_of_maxs,
   'TotalFloodedArea':totals_df['FloodedArea'],'%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs'],
    '%Diff_FloodedArea_fromSP_formatted':percent_diffs_df['percent_diff_formatted'],
    'Abs%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs_abs'],'UrbanFloodedArea':totals_df_urban['FloodedArea'],
  '%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs'] ,
  '%Diff_UrbanFloodedArea_fromSP_formatted':percent_diffs_df_urban['percent_diff_formatted'],
    'Abs%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs_abs'], 
    'WorstCaseDepth_ncells': worst_case_method_depth['counts'].tolist(),
    'WorstCaseVelocity_ncells': worst_case_method_velocity['counts'].tolist(), 'colour':colours_df['colour']}) 

In [148]:
cluster_results

,Cluster_num,MaxRainfallIntensity,MaxRainfallIntensityMinute,TotalFloodedArea,%Diff_FloodedArea_fromSP,%Diff_FloodedArea_fromSP_formatted,Abs%Diff_FloodedArea_fromSP,UrbanFloodedArea,%Diff_UrbanFloodedArea_fromSP,%Diff_UrbanFloodedArea_fromSP_formatted,Abs%Diff_UrbanFloodedArea_fromSP,WorstCaseDepth_ncells,WorstCaseVelocity_ncells,colour
0,6h_feh_sp,0.201255,180,1.701144,0.00,,0.00,0.575595,0.00,,0.00,658,6203,black
1,6h_sp_fl_0.1,0.232031,36,1.636911,-3.78,-3.78%,3.78,0.540120,-6.16,-6.16%,6.16,193341,308382,darkblue
2,6h_sp_fl_0.2,0.243904,72,1.635671,-3.85,-3.85%,3.85,0.514048,-10.69,-10.69%,10.69,77643,114817,darkblue
3,6h_sp_fl_0.3,0.254309,108,1.654627,-2.73,-2.73%,2.73,0.523471,-9.06,-9.06%,9.06,6257,44945,paleturquoise
4,6h_sp_fl_0.4,0.262907,144,1.678387,-1.34,-1.34%,1.34,0.532838,-7.43,-7.43%,7.43,10468,49478,paleturquoise
5,6h_sp,0.269191,180,1.727952,1.58,+1.58%,1.58,0.581404,1.01,+1.01%,1.01,432204,393317,grey
6,6h_sp_bl_0.6,0.292262,215,1.729529,1.67,+1.67%,1.67,0.551026,-4.27,-4.27%,4.27,19403,37834,indianred
7,6h_sp_bl_0.7,0.317888,251,1.746964,2.69,+2.69%,2.69,0.559568,-2.78,-2.78%,2.78,11388,29585,indianred
8,6h_sp_bl_0.8,0.343370,287,1.764415,3.72,+3.72%,3.72,0.567784,-1.36,-1.36%,1.36,32676,36465,darkred
9,6h_sp_bl_0.9,0.368805,323,1.775070,4.35,+4.35%,4.35,0.574021,-0.27,-0.27%,0.27,775704,545804,darkred


### Add the depth/velocity category breakdowns and hazard categories to this

In [149]:
dfs = [velocity_props, depth_props, velocity_props_urban, depth_props_urban, velocity_counts, depth_counts,
          velocity_counts_urban, depth_counts_urban,hazard_counts, hazard_props]
suffixes = ['_propcells', '_propcells','_propcells_urban','_propcells_urban','_countcells','_countcells','_countcells_urban','_countcells_urban',
'_numcells', '_propcells']

for num, df in enumerate(dfs):
    # Reformat the dataframe
    df = df.set_index('index').T
    # Add the correct suffix to the column names
    df = df.add_suffix(suffixes[num]) 
    # Add Cluster_num column for joining
    df['Cluster_num'] = df.index#
    # Join to cluster results dataframe
    cluster_results = pd.merge(cluster_results,  df, how="outer", on = 'Cluster_num')
    
cluster_results = pd.merge(cluster_results, hazard_cat_changes,  how="outer", on = 'Cluster_num')    

### Save to file

In [150]:
cluster_results.to_csv("Data/allclusters_summary.csv", index=False)

### Delete tiff files (as these aren't used again and take up a lot of space)

In [156]:
for method in short_ids:
    print(method)
    if method != '6h_feh_sp':
        os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_cat_difference.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_classified.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_posneg.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_classified.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_posneg.tif".format(method)) 
        
    os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_classified.tif".format(method)) 
    os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_classified.tif".format(method)) 
    os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_classified.tif".format(method)) 